In [177]:
import psycopg2
import psycopg2.extras
import csv
from pprint import pprint as pp
from tabulate import tabulate

In [178]:
conn = psycopg2.connect("host=localhost port=5432 dbname=odscourse user=postgres password=mi")
cursor = conn.cursor()

In [179]:
def fetch_all(cursor):
    colnames = [desc[0] for desc in cursor.description]
    records = cursor.fetchall()
    return [{colname:value for colname, value in zip(colnames, record)} for record in records]

In [180]:
cursor.execute("SELECT * FROM mlboot LIMIT 5")
records = cursor.fetchall()
print(records)

[(0, 18393, 2, 168.0, 62.0, 110, 80, 1, 1, False, False, True, False), (1, 20228, 1, 156.0, 85.0, 140, 90, 3, 1, False, False, True, True), (2, 18857, 1, 165.0, 64.0, 130, 70, 3, 1, False, False, False, True), (3, 17623, 2, 169.0, 82.0, 150, 100, 1, 1, False, False, True, True), (4, 17474, 1, 156.0, 56.0, 100, 60, 1, 1, False, False, False, False)]


In [181]:
'''Сколько мужчин и женщин представлено в этом наборе данных?'''
cursor.execute(
    """
    SELECT gender, COUNT(*)
        FROM mlboot
        GROUP BY gender
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+----------+---------+
|   gender |   count |
|----------+---------|
|        1 |   45530 |
|        2 |   24470 |
+----------+---------+


In [182]:
'''Кто в среднем реже указывает, что употребляет алкоголь – мужчины или женщины?'''
cursor.execute(
    """
    SELECT gender, AVG(alco::int)
        FROM mlboot
        GROUP BY gender
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+----------+-----------+
|   gender |       avg |
|----------+-----------|
|        1 | 0.0254997 |
|        2 | 0.106375  |
+----------+-----------+


In [183]:
'''Во сколько раз (округленно, round) процент курящих среди мужчин больше, чем процент
    курящих среди женщин (по крайней мере, по этим анкетным данным)?'''
cursor.execute(
    """
    SELECT DISTINCT ROUND((SELECT AVG(smoke::int) 
    FROM mlboot 
    WHERE gender = 2)/(SELECT AVG(smoke::int) 
    FROM mlboot 
    WHERE gender = 1)) AS ROUND 
    FROM mlboot;
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+---------+
|   round |
|---------|
|      12 |
+---------+


In [190]:
""" На сколько месяцев (примерно) отличаются медианные значения
    возраста курящих и некурящих? """

cursor.execute(
    """
    SELECT DISTINCT (
        (SELECT AVG(age::int)/30 FROM mlboot WHERE smoke='0') - 
        (SELECT AVG(age::int)/30 FROM mlboot WHERE smoke='1')
    )::int AS difference
    FROM mlboot
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+--------------+
|   difference |
|--------------|
|           14 |
+--------------+


In [138]:
""" 1)Создайте новый признак age_years – возраст в годах, округлив до целых (round). 
    Для данного примера отберите курящих мужчин от 60 до 64 лет включительно
    2)Создайте 2 подвыборки курящих мужчин возраста от 60 до 64 лет включительно: 
    первая с ap_hi строго меньше 120 мм рт.ст. и концентрацией холестерина – 4 ммоль/л, 
    вторая – с ap_hi от 160 (включительно) до 180 мм рт.ст. (не включительно) и холестерином – 8 ммоль/л.
    
    Во сколько раз (округленно) отличаются доли больных людей (согласно целевому признаку,cardio) в этих двух подвыборках? 
"""
cursor.execute(
    """
SELECT DISTINCT (
    ROUND((
    (SELECT AVG(cardio::int)
    FROM (
    SELECT * FROM mlboot
    WHERE 60 <= ROUND(age/365) AND ROUND(age/365) <= 64 AND gender=2 AND smoke=true
    ) AS qq
    WHERE cholesterol = 3 AND ap_hi >= 160 AND ap_lo < 180 AND gender=2 AND smoke=true)
    /
    (SELECT AVG(cardio::int)
    FROM (
    SELECT * FROM mlboot
    WHERE 60 <= ROUND(age/365) AND ROUND(age/365) <= 64 AND gender=2 AND smoke=true
    ) AS qq
    WHERE cholesterol = 1 AND ap_hi < 120 AND gender=2 AND smoke=true)))
    ) AS difference
    FROM mlboot;
    
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+--------------+
|   difference |
|--------------|
|            3 |
+--------------+


In [118]:
"""Постройте новый признак – BMI (Body Mass Index). 
Для этого надо вес в килограммах поделить на квадрат роста в метрах. 
Нормальными считаются значения BMI от 18.5 до 25. 
Выберите верные утверждения.

1)Медианный BMI по выборке превышает норму"""

cursor.execute(
    """
    SELECT AVG(weight / (height / 100) ^ 2::int) AS bmi FROM mlboot
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))


+---------+
|     bmi |
|---------|
| 27.5565 |
+---------+


In [119]:
'''2)У женщин в среднем BMI ниже, чем у мужчин'''

cursor.execute(
    """
    SELECT DISTINCT
        (SELECT AVG(weight / (height / 100) ^ 2) FROM mlboot WHERE gender=1) AS women,
        (SELECT AVG(weight / (height / 100) ^ 2) FROM mlboot WHERE gender=2) AS men
    FROM mlboot
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+---------+---------+
|   women |     men |
|---------+---------|
| 27.9876 | 26.7544 |
+---------+---------+


In [120]:
'''3)У здоровых в среднем BMI выше, чем у больных'''

cursor.execute(
    """
    SELECT DISTINCT
        (SELECT AVG(weight / (height / 100) ^ 2) FROM mlboot WHERE cardio::int=0) AS healthy,
        (SELECT AVG(weight / (height / 100) ^ 2) FROM mlboot WHERE cardio::int=1) AS unhealthy
    FROM mlboot
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+-----------+-------------+
|   healthy |   unhealthy |
|-----------+-------------|
|   26.5482 |     28.5661 |
+-----------+-------------+


In [140]:
'''4)В сегменте здоровых и непьющих мужчин в среднем BMI ближе к норме, чем в сегменте здоровых и непьющих женщин'''

cursor.execute(
    """
    SELECT DISTINCT
        (SELECT AVG(weight / (height / 100) ^ 2) FROM mlboot
         WHERE cardio::int=0 AND alco::int=0 AND gender=2) AS bmi_men,
        (SELECT AVG(weight / (height / 100) ^ 2) FROM mlboot
         WHERE cardio::int=0 AND alco::int=0 AND gender=1) AS bmi_women
    FROM mlboot
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+-----------+-------------+
|   bmi_men |   bmi_women |
|-----------+-------------|
|   25.8726 |     26.8454 |
+-----------+-------------+


In [137]:
'''Отфильтруйте следующие сегменты пациентов (считаем это ошибками в данных)

1)указанное нижнее значение артериального давления строго выше верхнего
2)рост строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили 
3)вес строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили

Сколько процентов данных (округленно, round) мы выбросили?'''

cursor.execute(
    """
SELECT DISTINCT(
        100 - ((
        SELECT DISTINCT COUNT(*) * 100
        FROM mlboot
        WHERE ap_hi >= ap_lo AND
              height >= 150 AND height <= 180 AND
              weight >= 51 AND weight <= 108
        ) / (
        SELECT COUNT(*) FROM mlboot
        ))
    ) AS extra
    FROM mlboot
    
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+---------+
|   extra |
|---------|
|      10 |
+---------+
